In [2]:
import pandas as pd
from glob import glob
import datetime
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [3]:
df=pd.read_csv('CoinText_final.csv')
df=df.drop('Unnamed: 0',axis=1)


## 情緒分數轉換

In [4]:
# 讀取檔案
files = glob('CoinText_final.csv')

# 合併檔案
df = pd.concat((pd.read_csv(file, usecols=['CoinDate','CoinText'], dtype={ 'CoinDate': str, 'CoinText':str}) for file in files))
df.reset_index()

# 建立日期迴圈
start = datetime.datetime.strptime("2022/1/01", "%Y/%m/%d")
end = datetime.datetime.strptime("2022/12/31", "%Y/%m/%d")
date_generated = pd.date_range(start, end)


# 載入模型和tokenizer
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

#建立情緒分數清單
CionScore=[]
Cion_score=0
CoinDate_n=[]
PCion_score=0
i=0

# 設定要分類的標籤
labels = ['positive', 'negative', 'neutral']



for context_date in date_generated:
    #篩選時間
    mask1 = (df["CoinDate"] == str(context_date.strftime("%Y/%m/%d")))
    if len(df[mask1])==0:      
        CionScore.append(PCion_score*0.99)
        CoinDate_n.append(context_date.strftime("%Y/%m/%d"))
    else:
        for Context in df[mask1]['CoinText']:
            # 將文本轉換為tokens並加入特殊tokens
            inputs = tokenizer(str(Context), return_tensors='pt', padding=True, truncation=True)

            # 進行預測
            outputs = model(**inputs)
            predicted_scores = torch.softmax(outputs.logits, dim=1)

            # 挑選預測中得分最高的情緒類別
            max_score_idx = torch.argmax(predicted_scores)
            max_score_label = labels[max_score_idx]
            max_score = predicted_scores[0][max_score_idx]

            if max_score_label=='positive':
                Cion_score+=float(max_score)
            elif max_score_label=='negative':
                Cion_score+=float(-1*max_score)
            elif max_score_label=='neutral':
                Cion_score+=0
        CoinDate_n.append(context_date.strftime("%Y/%m/%d"))
        PCion_score=Cion_score/len(df[mask1])
        CionScore.append(PCion_score)
        Cion_score=0
        print(context_date.strftime("%Y/%m/%d"))
    i+=1
    if ((i%100)==0) &(i>0):
        lo={'CionScore':CionScore,'CoinDate':CoinDate_n}
        dfz = pd.DataFrame(lo)
        dfz.to_csv('Data_score/CionScore_'+str(i)+'.csv')


lo={'CionScore':CionScore,'CoinDate':CoinDate_n}
dfz = pd.DataFrame(lo)
dfz.to_csv('Data_score/CionScore_final.csv')


2022-01-01
2022-01-03
2022-01-04
2022-01-05
2022-01-06
2022-01-07
2022-01-08
2022-01-09
2022-01-10
2022-01-11
2022-01-12
2022-01-13
2022-01-14
2022-01-16
2022-01-17
2022-01-18
2022-01-19
2022-01-20
2022-01-21
2022-01-23
2022-01-24
2022-01-25
2022-01-26
2022-01-27
2022-01-28
2022-01-29
2022-01-30
2022-01-31
2022-02-01
2022-02-02
2022-02-03
2022-02-04
2022-02-06
2022-02-07
2022-02-08
2022-02-09
2022-02-10
2022-02-11
2022-02-13
2022-02-14
2022-02-15
2022-02-16
2022-02-17
2022-02-18
2022-02-20
2022-02-21
2022-02-22
2022-02-23
2022-02-24
2022-02-25
2022-02-26
2022-02-27
2022-02-28
2022-03-01
2022-03-02
2022-03-03
2022-03-04
2022-03-06
2022-03-07
2022-03-08
2022-03-09
2022-03-10
2022-03-11
2022-03-12
2022-03-13
2022-03-14
2022-03-15
2022-03-16
2022-03-17
2022-03-18
2022-03-20
2022-03-21
2022-03-22
2022-03-23
2022-03-24
2022-03-25
2022-03-27
2022-03-28
2022-03-29
2022-03-30
2022-03-31
2022-04-01
2022-04-03
2022-04-04
2022-04-05
2022-04-06
2022-04-07
2022-04-08
2022-04-10
2022-04-11
2022-04-12

## 合併檔案

In [19]:
import pandas as pd
from glob import glob
 
files =['CionScore_final.csv','Data_final/BTC-USD.csv']
 
df = pd.concat((pd.read_csv(file) for file in files), axis='columns').drop(['Date','Adj Close','Unnamed: 0'],axis=1)
df.to_csv('Data_final/BTC_final.csv')
df.head()

,CionScore,CoinDate,Open,High,Low,Close,Volume
0,-0.224409,2014-10-01,387.427002,391.378998,380.779999,383.614990,26229400.0
1,0.162439,2014-10-02,383.988007,385.497009,372.946014,375.071991,21777700.0
2,-0.136249,2014-10-03,375.181000,377.695007,357.859009,359.511993,30901200.0
3,-0.134887,2014-10-04,359.891998,364.487000,325.885986,328.865997,47236500.0
4,-0.134887,2014-10-05,328.915985,341.800995,289.295990,320.510010,83308096.0


In [6]:
df=pd.read_csv('Data_original/CoinText_3600.csv')
for i in range(36):
    f= open("CoinContext/CoinContext"+df['CoinDate'][i]+"-"+str(i)+".txt","w+")
    f.write(df['CoinText'][i])
    f.close()


In [10]:
df2=df[['CoinDate','CoinText']][:36]
df2.to_csv('CoinText_all.csv')